In [3]:
import numpy as np
import math
from IPython.core.debugger import set_trace

lowerBound, upperBound = -30, 30
x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
N = len(x)


def f(x):
    #Sphere
    # x in interval [-30, -30]
    #Sum = 0
    #for i in range(0,N):
    #    Sum = Sum + (x[i])**2

    #Rosenbrock
    #za epsilon = 10e-30 dao nulu
    # x in interval [-30, -30]
    Sum = 0
    for i in range(0, N-1):
        Sum = Sum + 100 * ((x[i])**2 - x[i+1])**2 + (x[i] - 1)**2
        
    #Ackley
    # x in interval [-5, -10]
    #Sum1, Sum2 = 0, 0
    #for i in range(0,N):
    #    Sum1 = Sum1 + (x[i])**2
    #    Sum2 = Sum2 + math.cos(2*math.pi*x[i])
    #Sum= -20.0*math.exp(-0.2*math.sqrt((1/N)*(Sum1)))-math.exp((1/N) * Sum2) + math.e + 20
    return Sum

l = [lowerBound for i in range(10)]
u = [upperBound for i in range(10)]

In [4]:
def _e(index,size):
    arr = np.zeros(size)
    arr[index] = 1.0
    return arr

In [5]:
def Delta(x):
    Sum = 0
    #no general, only box constraints
    for i in range(0,N):
        Sum = Sum + max(x[i]-u[i],0) + max(l[i]-x[i], 0)
    return Sum

In [6]:
def better(x,y):
    Dx = Delta(x)
    Dy = Delta(y)
    if(Dx < Dy):
        return True
    elif(Dx == Dy and f(x) < f(y)):
        return True
    else:
        return False

In [7]:
def sortSimplex(s):
    for i in range(len(s)):
        # Find the minimum element in remaining
        # unsorted array
        min_idx = i
        for j in range(i+1, len(s)):
            if better(s[j],s[min_idx]):
                min_idx = j
        # Swap the found minimum element with
        # the first element       
        for k in range(0,N):
            s[i][k], s[min_idx][k] = s[min_idx][k], s[i][k]
    
    return s

In [8]:
def centroid(s,argWorst):
    return np.mean(np.delete(s, argWorst), axis = 0)

In [9]:
def SimplexLocalSearch(option,x,Epsilon,Lambda,M,Alpha=1,Gamma=2,Beta=-0.5,no_improv_break = 100):
    N = len(x)
    s = np.empty([N+1,N])
    s[0] = x
    for j in range(1,N+1):
        s[j] = x + Lambda*(u[j-1]-l[j-1])*_e(j-1,N)
    
    k = 0
    
    s = sortSimplex(s)
    prev_best = f(s[0])
    no_improv = 0
    while abs(f(s[0]) - f(s[-1])) > Epsilon and k < M:
        best = f(s[0])
        
        # break after no_improv_break iterations with no improvement
        #print('...best so far:', best)

        if best < prev_best - Epsilon:
            no_improv = 0
            prev_best = best
        else:
            no_improv += 1

        if no_improv >= no_improv_break:
            return s[0],k,s

        r = (1+Alpha)*centroid(s,-1) - Alpha*s[-1]
        if(better(s[0],r) and better(r,s[-2])):
            s[-1] = copy.copy(r)
        elif(better(r,s[0])):
            e = (1+Gamma)*centroid(s,-1)-Gamma*s[-1]
            if(better(e,r)):
                s[-1] = copy.copy(e)
            else:
                s[-1] = copy.copy(r)
        else:
            c = (1+Beta)*centroid(s,-1)-Beta*s[-1]
            if(better(c,s[-1])):
                s[-1] = copy.copy(c)
            else:
                if(option == 1): 
                    #print(f(s[0]))
                    return s[0],k, s
                for j in range(0,N+1):
                    s[j] = (s[j] + s[0])/2

        k = k + 1
        s = sortSimplex(s)
    return s[0], k, s

In [10]:
import copy
def DirectionalEscape(Epsilon,EpsilonApostrophe,Lambda,M, Gamma = 2,Beta=-0.5):
    k = 0
    it = 0
    
    x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
    N = len(x)
    s = np.empty([N+1,N])
    s[0] = x
    for j in range(1,N+1):
        s[j] = x + Lambda*(u[j-1]-l[j-1])*_e(j-1,N)
    
    while(k < M):
        
        temp, _k, s = SimplexLocalSearch(1,x,Epsilon,Lambda,M-k)
        x = copy.copy(temp)
        
        k = k + _k
        try: xBest
        except NameError: xBest = None
        
        if xBest is None or f(x) < f(xBest):
            xBest = copy.copy(x)
            temp, _k, _ = SimplexLocalSearch(2,x,EpsilonApostrophe,Lambda,M-k)
            if(f(temp) < f(xBest)):
                xBest = copy.copy(temp)
            k = k + _k
            print("***NEW BEST VALUE*** iteration: ", k, "| best found value: ", f(xBest))
            
        if xBest is not None and k//1000 > it:
            it = k//1000
            print("iteration:", it * 1000, "| best found value: ", f(xBest))
            
            
        
        try: xWorst
        except NameError: xWorst = None
        while xWorst is None or better(x,xWorst):
            xWorst = copy.copy(x)
            
            s = sortSimplex(s)
            x = Gamma*s[0] + (1-Gamma) * centroid(s,0)
            s[0] = copy.copy(x)
            s = sortSimplex(s)
            
            
        
        
    return xBest

In [ ]:
Lambda = 10
M = 100000
Epsilon = 0
#set_trace()
EpsilonApostrophe = 0
print(DirectionalEscape(Epsilon,EpsilonApostrophe,Lambda,M))

***NEW BEST VALUE*** iteration:  314 | best found value:  327.4492579882804
iteration: 1000 | best found value:  327.4492579882804
iteration: 2000 | best found value:  327.4492579882804
iteration: 3000 | best found value:  327.4492579882804
